In [6]:
import geopandas as gpd
import pandas as pd
import math
import ee
import geemap

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-curuai')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import image collections

Landsat 7 - PY6S

In [4]:
landsat7 = ee.ImageCollection("projects/ee-curuai/assets/Py6S/LD7/ld7_py6s")\
            .select([ 'B1', 'B2', 'B3', 'B4', 'B5', 'B7'])
print(ee.Date(landsat7.first().get('system:time_start')).format().getInfo())
print(ee.Date(landsat7.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

2013-05-12T13:44:05
2017-09-28T13:50:48


Landsat 8 PY6S

In [5]:
landsat8 = (ee.ImageCollection("projects/ee-curuai/assets/Py6S/LD8/ld8_py6s")
            .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7']))
print(ee.Date(landsat8.first().get('system:time_start')).format().getInfo())
print(ee.Date(landsat8.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

2013-05-20T13:50:14
2017-09-27T13:54:12


# Transform into remote sensing reflectance and sunglint correction

In [48]:
def deglint (img):
  '''#dividir imagem corrigida por pi
  Rrs_sat_ac = Rsat_ac / pi
  fazer deglint
  Rrs_sat_ac_deglint(VNIR) = Rrs_sat_ac (VNIR) − Rrs_sat_ac (SWIR)
  correção no artigo INPE CURUAI'''
  Rrs = img.divide(math.pi)
  deglint = Rrs.select(['blue_mean','green_mean','red_mean','nir_mean','swir1','swir2'])\
    .subtract(Rrs.select('swir1'))

  return (deglint.updateMask(deglint.select(['red_mean']).gt(0)).updateMask(deglint.select(['blue_mean']).gt(0)).updateMask(deglint.select(['green_mean']).gt(0)).updateMask(deglint.select(['nir_mean']).gt(0))
          .copyProperties(img,['system:time_start','CLOUD_COVER',"system:index"]))

## Padronize band names

In [49]:
name_bands = ['blue_mean','green_mean','red_mean','nir_mean','swir1','swir2']

Landsat 7

In [50]:
#renomear bandas

ld7 = landsat7.map(lambda img: img.rename(name_bands)).map(deglint)
ld7.first()

Landsat 8

In [51]:
ld8 = landsat8.map(lambda img: img.rename(name_bands)).map(deglint)
ld8.first()

# Import Model

In [52]:
classifier = ee.Classifier.load('projects/ee-curuai/assets/RF_regressor')
classifier

In [58]:
predictors = ['blue_mean', 'green_mean','red_mean', 'nir_mean']
ld7_classified = ld7.select(predictors).map(lambda img: img.classify(classifier=classifier))
ld7_classified

In [63]:
ld8_classified = ld8.select(predictors).map(lambda img: img.classify(classifier=classifier))
ld8_classified

In [65]:
Map = geemap.Map(basemap='HYBRID')
roi_points = ee.FeatureCollection('projects/ee-curuai/assets/curuai_points')
Map.centerObject(roi_points,10)
# imgld7 = ld7.sort('system:time_start',False).first()
# imgLD8 = ld8.sort('CLOUD_COVER').first()

# Map.addLayer(imgld7, {'bands':['red_mean','green_mean','blue_mean'], 'min':0,'max':0.02}, str(imgld7.get('system:index').getInfo()))
# Map.addLayer(imgLD8, {'bands':['red_mean','green_mean','blue_mean'], 'min':0,'max':0.02}, str(imgLD8.get('system:index').getInfo()))

Map.addLayer(ld7_classified.mean(), {'palette':['#ffffb2','#fecc5c','#fd8d3c','#e31a1c'], 'min':0,'max':150}, 'classification LD7')
Map.addLayer(ld8_classified.mean(), {'palette':['#ffffb2','#fecc5c','#fd8d3c','#e31a1c'], 'min':0,'max':150}, 'classification LD8')


Map.addLayer(roi_points, {'color':'darkred'}, 'Data polygon');
Map

Map(center=[-2.184599717160098, -55.49304633852027], controls=(WidgetControl(options=['position', 'transparent…